In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow
import pandas as pd

In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [ ]:
conv_base.summary()

In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
conv_base.trainable = False

In [ ]:
# Load CSV file
train_csv = '../../../../Dataset/Plant Disease Detection/train/_classes.csv'  
train_df = pd.read_csv(train_csv)

valid_csv = '../../../../Dataset/Plant Disease Detection/valid/_classes.csv'  
valid_df = pd.read_csv(valid_csv)

train_df.columns = train_df.columns.str.strip()
valid_df.columns = valid_df.columns.str.strip()

# Create ImageDataGenerators 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create a DataFrame with file paths and labels
train_df['filename'] = '../../../../Dataset/Plant Disease Detection/train/' + train_df['filename']  
train_df['label'] = train_df[['bercak_daun', 'defisiensi_kalsium', 'hangus_daun', 'hawar_daun', 'mosaik_vena_kuning', 'virus_kuning_keriting']].idxmax(axis=1)

valid_df['filename'] = '../../../../Dataset/Plant Disease Detection/valid/' + valid_df['filename']  
valid_df['label'] = valid_df[['bercak_daun', 'defisiensi_kalsium', 'hangus_daun', 'hawar_daun', 'mosaik_vena_kuning', 'virus_kuning_keriting']].idxmax(axis=1)


# Create ImageDataGenerator using flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Create ImageDataGenerator using flow_from_dataframe
valid_generator = test_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,epochs=5,validation_data=valid_generator)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the new image file
class_names = ['bercak_daun', 'defisiensi_kalsium', 'hangus_daun', 'hawar_daun', 'mosaik_vena_kuning', 'virus_kuning_keriting']
new_image_path = '../../../../Dataset/Plant Disease Detection/test/bercak_daun--54-_JPG.rf.29291b25d0990a1b964bb68382f1a986.jpg'  # Change this to the path of your new image
new_img = image.load_img(new_image_path, target_size=(150, 150))  # Adjust target_size as needed
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0) / 255.0  # Normalize the image

# Make predictions
predictions = model.predict(new_img_array)

# Display the predicted probabilities for each class
print(predictions)

predicted_class = np.argmax(predictions)
# Get the final predicted class name
predicted_class_name = class_names[predicted_class]

predicted_class_name

In [ ]:
# Save the model
model.save("../models/plantDisease_model.h5")

In [1]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('../models/plantDisease_model')

In [2]:
loaded_model.save("../models/plantDisease_model.h5")

c:\Users\MIS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
loaded_model = load_model('../models/plantDisease_model.h5')

In [4]:
from tensorflow.keras.preprocessing import image
import numpy as np
# Load the new image file
class_names = ['brown spots', 'deficiency calcium', 'xanthomonas', 'stemphylium solani', 'mosaic vena kuning', 'virus king keriting']
new_image_path = './1.jpg'  # Change this to the path of your new image
new_img = image.load_img(new_image_path, target_size=(150, 150))  # Adjust target_size as needed
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0) / 255.0  # Normalize the image

# Make predictions
predictions = loaded_model.predict(new_img_array)

# Display the predicted probabilities for each class
print(predictions)

predicted_class = np.argmax(predictions)
# Get the final predicted class name
predicted_class_name = class_names[predicted_class]

predicted_class_name

1/1 [==============================] - 1s 1s/step
[[9.9999416e-01 6.0135599e-09 3.2821441e-07 2.1230221e-06 8.8866986e-10
  3.3608057e-06]]


'brown spots'